In [1]:
import torch
import shutil
from transformers import LlamaTokenizer
from ipex_llm.transformers.qlora import PeftModel
from ipex_llm.transformers import AutoModelForCausalLM

base_model = "meta-llama/Llama-2-7b-hf"
adapter = "2024-05-09_18-26-55"

adapter_path = "./" + adapter + "/results/checkpoint-250/"
output_path = "./merged/" + adapter + "/"

tokenizer = LlamaTokenizer.from_pretrained(base_model)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model,
    # load_in_low_bit="nf4", # should load the orifrom transformers import LlamaTokenizergnal model
    torch_dtype=torch.float16,
    device_map={"": "cpu"},
)

lora_model = PeftModel.from_pretrained(
    base_model,
    adapter_path,
    device_map={"": "cpu"},
    torch_dtype=torch.float16,
)

# merge weights - new merging method from peft
lora_model = lora_model.merge_and_unload()

lora_model.train(False)

lora_model_sd = lora_model.state_dict()
deloreanized_sd = {
    k.replace("base_model.model.", ""): v
    for k, v in lora_model_sd.items()
    if "lora" not in k
}

base_model.save_pretrained(output_path, state_dict=deloreanized_sd)
tokenizer.save_pretrained(output_path)
shutil.copyfile("./" + adapter + "/parameters.json", output_path + "parameters.json")

/home/gregor/venv/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: ''If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2024-05-13 08:39:36,811 - INFO - intel_extension_for_pytorch auto imported
/home/gregor/venv/lib/python3.9/site-packages/bitsandbytes/cextension.py:31: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/home/gregor/venv/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2024-05-13 08:39:46,241 - INFO - We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


'./merged/2024-05-09_18-26-55/parameters.json'